In [3]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

In [4]:
case class Message(source: Long, value: (Double, Double))
case class Node(typeOfNode: Int, neighbors: Int, distribution: Array[Double], messages: List[Message], firstDestination: Long)

defined class Message
defined class Node


In [5]:
val fileName = "../data/input.txt"
var lines = sc.textFile(fileName).collect()
var variablesNumber: Long = 0
var factorsNumber: Long = 0
variablesNumber = lines(0).split(" ")(0).toLong
factorsNumber = lines(0).split(" ")(1).toLong
var tempNodes: Array[(Long, Node)] = new Array[(Long, Node)]((variablesNumber+factorsNumber).toInt) // (id, (type, leaf, distribution))
for (a <- 0 to (variablesNumber+factorsNumber - 1).toInt){
    tempNodes(a) = (a.toLong, Node(0,0, null, List[Message](), -1))
}
var tempEdges: Array[Edge[(Long)]] = new Array[Edge[(Long)]](((variablesNumber+factorsNumber-1)*2).toInt)
var cont = 0
for (a <- 1 to lines.size-1){
    if(a%2 == 1){
        val factorNode = variablesNumber + ((a-1)/2).toLong
        var pos = 0
        for (variableNode <- lines(a).split(" ")){
            tempEdges(cont) = Edge(variableNode.toLong, factorNode, 0)
            cont = cont + 1
            tempEdges(cont) = Edge(factorNode, variableNode.toLong, pos)
            cont = cont + 1
            pos = pos + 1
        } 
    }else{
        val distribution:  Array[Double] = lines(a).split(" ").map(_.toDouble)
        val factorNode = (variablesNumber + ((a-1)/2).toLong).toInt
        tempNodes(factorNode) = (factorNode, Node(1, 0, distribution, List[Message](), -1))
    }
}

fileName = ../data/input.txt
lines = Array(5 5, 0 1, 0.25 0.25 0.25 0.25, 1 2 3, 0.16706443914081145 0.01909307875894988 0.18138424821002386 0.1431980906921241 0.02386634844868735 0.15513126491646778 0.07159904534606205 0.2386634844868735, 2, 0.4 0.6, 3 4, 0.20 0.30 0.40 0.10, 3, 0.25 0.75)
variablesNumber = 5
factorsNumber = 5
variablesNumber = 5
factorsNumber = 5
tempNodes = Array((0,Node(0,0,null,List(),-1)), (1,Node(0,0,null,List(),-1)), (2,Node(0,0,null,List(),-1)), (3,Node(0,0,null,List(),-1)), (4,Node(0,0,null,List(),-1)), (5,Node(1,0,[D@147c94e,List(),-1)), (6,Node(1,0,[D@19b574ba,List(),-1)), (7,Node(1,0,[D@45c7fd5d,List(),-1)), (8,Node(1,0,[D@aa38070,List(),-1)), (9,Node(1,0,[D@359135d3,List(),-1)))


tempEdges:...


Array((0,Node(0,0,null,List(),-1)), (1,Node(0,0,null,List(),-1)), (2,Node(0,0,null,List(),-1)), (3,Node(0,0,null,List(),-1)), (4,Node(0,0,null,List(),-1)), (5,Node(1,0,[D@147c94e,List(),-1)), (6,Node(1,0,[D@19b574ba,List(),-1)), (7,Node(1,0,[D@45c7fd5d,List(),-1)), (8,Node(1,0,[D@aa38070,List(),-1)), (9,Node(1,0,[D@359135d3,List(),-1)))

In [6]:
val nodes: RDD[(VertexId, Node)] = sc.parallelize(tempNodes)
val edges: RDD[Edge[Long]] = sc.parallelize(tempEdges)
val graph = Graph(nodes, edges)

nodes = ParallelCollectionRDD[2] at parallelize at <console>:39
edges = ParallelCollectionRDD[3] at parallelize at <console>:40
graph = org.apache.spark.graphx.impl.GraphImpl@409347b1


org.apache.spark.graphx.impl.GraphImpl@409347b1

In [7]:
val inDegrees: VertexRDD[Int] = graph.inDegrees
//leaves.collect().foreach(println)
val graphWithLeaves = graph.joinVertices(inDegrees){
 case (id, node, neighbors) =>  Node(node.typeOfNode, neighbors, node.distribution, node.messages, node.firstDestination)
}
//graphWithLeaves.vertices.collect().foreach(println)

inDegrees = VertexRDDImpl[19] at RDD at VertexRDD.scala:57
graphWithLeaves = org.apache.spark.graphx.impl.GraphImpl@38b80b60


org.apache.spark.graphx.impl.GraphImpl@38b80b60

In [20]:
def getValueFromDistribution(distribution: Array[Double], pos: Array[Int]): Double = {
    -1
}

getValueFromDistribution: (distribution: Array[Double], pos: Array[Int])Double


In [11]:
def computeProductOfMessagesForVariableNode(sourceNode: Node): (Double, Double) = {
    var product: (Double, Double) = (1,1)
    for(msg <- sourceNode.messages){
        product = (product._1*msg.value._1, product._2*msg.value._2)
    }
    product
}

computeProductOfMessagesForVariableNode: (sourceNode: Node)(Double, Double)


In [14]:
def computeMessageToFactor(sourceId: Long, sourceNode: Node, destinationId: Long): Message = {
    if(sourceNode.messages.size == sourceNode.neighbors - 1){
        val firstDestination = sourceNode.firstDestination
        if(firstDestination == destinationId)
            Message(sourceId, computeProductOfMessagesForVariableNode(sourceNode))
        else
            null
    }else if(sourceNode.messages.size == sourceNode.neighbors){
        val product: (Double, Double) = computeProductOfMessagesForVariableNode(sourceNode)
        if(destinationId == sourceNode.firstDestination)
            null
        else{
            for(msg <- sourceNode.messages){
                if(msg.source == destinationId){
                    Message(sourceId, (product._1/msg.value._1, product._2/msg.value._2))
                }
            }
            null //should not happen
        }
    }else{
        null
    }
}

computeMessageToFactor: (sourceId: Long, sourceNode: Node, destinationId: Long)Message


In [21]:
def computeMessageToVariable(sourceId: Long, sourceNode: Node, destinationId: Long): Message = {
    return null
}

computeMessageToVariable: (sourceId: Long, sourceNode: Node, destinationId: Long)Message


In [68]:
def mergeMsg(msg1: List[Message], msg2: List[Message]): List[Message] = msg1:::msg2

mergeMsg: (msg1: List[Message], msg2: List[Message])List[Message]


In [9]:
def vprog(vertexId: VertexId, value: Node, message: List[Message]): Node = {
    if(message.head.source == -1){
        Node(value.typeOfNode, value.neighbors, value.distribution, value.messages, value.firstDestination)
    }
    else{
       
        val newMessages = value.messages:::message
        var firstDestination: Long = -1
        if(value.messages.size == value.neighbors && message.size == 1)
            firstDestination = message.head.source
        Node(value.typeOfNode, value.neighbors, value.distribution, newMessages, firstDestination)
    }
}

vprog: (vertexId: org.apache.spark.graphx.VertexId, value: Node, message: List[Message])Node


In [23]:
//tofix
def sendMsg(triplet: EdgeTriplet[Node, Long]): Iterator[(VertexId, List[Message])] = {
    val sourceId = triplet.srcId
    val sourceNode = triplet.srcAttr
    val destinationId = triplet.dstId
    if(sourceNode.messages.size == 0){
        if(sourceNode.neighbors == 1){
            if(sourceNode.typeOfNode == 0)
                Iterator((destinationId, List(Message(sourceId, (1,1)))))
            else
                Iterator((destinationId, List(Message(sourceId, (getValueFromDistribution(sourceNode.distribution, Array(0)), getValueFromDistribution(sourceNode.distribution, Array(1)))))))
        }else{
            Iterator.empty
        }
    }else{
        if(sourceNode.neighbors == 1){
            Iterator.empty
        }else{
            if(sourceNode.typeOfNode == 0)
                Iterator((destinationId, List(computeMessageToFactor(sourceId, sourceNode, destinationId))))
            else
                Iterator((destinationId, List(computeMessageToVariable(sourceId, sourceNode, destinationId))))
        }
        
    }
    
  
}

sendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[Node,Long])Iterator[(org.apache.spark.graphx.VertexId, List[Message])]


In [ ]:
val initialMsg = Message(-1, (-1, -1))
val maxIterations = (variablesNumber+factorsNumber)*2
val finalGraph = graph.pregel(initialMsg, maxIterations, EdgeDirection.Out)(vprog, sendMsg, mergeMsg)